In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('fraud_detection_bank_dataset.csv')
df.head()

,Unnamed: 0,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,...,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,targets
0,0,9,1354,0,18,0,1,7,9,0,...,0,0,0,1,1,0,0,0,49,1
1,1,0,239,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,55,1
2,2,0,260,0,4,0,3,6,0,0,...,0,0,0,1,1,0,0,0,56,1
3,3,17,682,0,1,0,0,8,17,0,...,0,1,0,1,1,0,0,0,65,1
4,4,1,540,0,2,0,1,7,1,0,...,0,0,0,1,1,0,0,0,175,1


In [3]:
print(len(df))

20468


In [4]:
df['targets'].value_counts()

0    15030
1     5438
Name: targets, dtype: int64

In [5]:
for col in df.columns:
    if (df[col] == 0).all():
        df = df.drop([col], axis=1)

In [6]:
num_rows = len(df.index)
low_information_cols = []

for col in df.columns:
    cnts = df[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]
    
    if top_pct > 0.97:
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col, top_pct*100))
        print(cnts)
        print()

col_27: 97.51808%
0     19960
1       355
2        96
3        26
4        16
5         9
8         2
18        1
6         1
7         1
9         1
Name: col_27, dtype: int64

col_28: 99.99511%
1    20467
0        1
Name: col_28, dtype: int64

col_39: 99.87786%
0    20443
1       21
2        1
3        1
7        1
9        1
Name: col_39, dtype: int64

col_40: 99.92671%
0      20453
1          4
3          3
5          2
13         2
4          1
724        1
102        1
12         1
Name: col_40, dtype: int64

col_45: 97.16631%
0      19888
1        331
2         99
3         43
4         33
5         23
7         11
8          8
6          7
9          6
11         3
12         3
16         2
15         2
21         1
18         1
24         1
232        1
10         1
13         1
45         1
14         1
46         1
Name: col_45, dtype: int64

col_46: 98.81767%
0     20226
1       121
2        49
3        25
4        12
6         9
5         5
7         3
8         3
10      

In [7]:
df = df.drop(low_information_cols, axis=1)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

df.drop('Unnamed: 0', axis=1, inplace=True)
#df = (df - df.mean()) / (df.max() - df.min())

X = df.drop(['targets'], axis=1)
y = df['targets']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [9]:
from sklearn.ensemble import RandomForestClassifier

rndf = RandomForestClassifier()
rndf.fit(X_train, y_train)
rndf.score(X_test, y_test)

0.9337241491613744

In [10]:
model = RandomForestClassifier(n_estimators=1, max_features='auto', max_depth=4, n_jobs=-1, random_state=5)
model.fit(X_train,y_train)

model.score(X_test, y_test)

0.8444878684253379

In [15]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=28)

clf.fit(
    X_train, y_train,
    eval_set=(X_test, y_test))

Learning rate set to 0.289711
0:	learn: 0.4522848	test: 0.4455555	best: 0.4455555 (0)	total: 24.3ms	remaining: 656ms
1:	learn: 0.3393796	test: 0.3321908	best: 0.3321908 (1)	total: 47ms	remaining: 611ms
2:	learn: 0.3017400	test: 0.2964750	best: 0.2964750 (2)	total: 68.6ms	remaining: 571ms
3:	learn: 0.2621794	test: 0.2577849	best: 0.2577849 (3)	total: 90.5ms	remaining: 543ms
4:	learn: 0.2463715	test: 0.2424007	best: 0.2424007 (4)	total: 113ms	remaining: 519ms
5:	learn: 0.2343891	test: 0.2309784	best: 0.2309784 (5)	total: 135ms	remaining: 496ms
6:	learn: 0.2275154	test: 0.2246621	best: 0.2246621 (6)	total: 157ms	remaining: 470ms
7:	learn: 0.2197436	test: 0.2183434	best: 0.2183434 (7)	total: 180ms	remaining: 451ms
8:	learn: 0.2141050	test: 0.2145186	best: 0.2145186 (8)	total: 209ms	remaining: 440ms
9:	learn: 0.2109616	test: 0.2122690	best: 0.2122690 (9)	total: 231ms	remaining: 415ms
10:	learn: 0.2050706	test: 0.2057646	best: 0.2057646 (10)	total: 253ms	remaining: 391ms
11:	learn: 0.1982573

In [16]:
clf.score(X_test, y_test)

0.9301416707376648